In [1]:
import numpy as np
import pandas as pd
import fastparquet
from collections import defaultdict
from timeit import Timer
import time
from IPython.core.display import HTML,display
#------------------------------------------------------------------------------

# Question 3 - NHANES Data

### a. Appending Nhanes Demographic Data

The four cohorts are:
- **Baby Boomers**: Born 1946 - 1964
- **Generation X**: Born 1965-1976
- **Millenials or Generation Y**: Born 1977-1995
- **Gen Z**: Born 1996 and after

**Meaning of variables:**  

**SEQN**: Respondent sequence number  
**RIDAGEYR**: Age in years of the participant at the time of screening  
**RIDRETH3**: Recode of reported race   
**RIAGENDR**: Gender  
**DMDEDUC2**: Education  
**DMDMARTL**: Marital Status  
**RIDSTATR**: Interview and examination status of the participant  
**SDMVPSU**: Masked variance unit pseudo-PSU variable for variance estimation  
**SDMVSTRA**: Masked variance unit pseudo-stratum variable for variance estimation  
**WTMEC2YR**: Full sample 2 year MEC exam weight  
**WTINT2YR**: Full sample 2 year interview weight

In [2]:
# Defining the SAS files to combine
nhanes = ['DEMO_G.XPT',
          'DEMO_H.XPT',
          'DEMO_I.XPT',
          'DEMO_J.XPT']

oral_dent = ['OHXDEN_G.XPT',
             'OHXDEN_H.XPT',
             'OHXDEN_I.XPT',
             'OHXDEN_J.XPT']

In [3]:
def year_def(demo_dent):
    """
    Defining the year based on database
    
    Parameters
    ----------
    demo_dent : list
    Gives the locations of the demographic data SAS files

    Returns
    -------
    year: int
    Gives out the year of the dataset
    """
    
    if('G' in demo_dent):
        year = 2011
    elif('H' in demo_dent):
        year = 2013
    elif('I' in demo_dent):
        year = 2015
    elif('J' in demo_dent):
        year = 2017
    return year  

In [4]:
def cohort_def(age,data):
    """
    Defined to cohort based on age 
    
    Parameters
    ----------
    age : The RIDAGEYR variables
    The age variables for unique SEQN
    
    demo_dent: list
    Gives the locations of the demographic data SAS files

    Returns
    -------
    check: string
    Returns the cohort based on year
    """
    
    year=year_def(data)
    if(age <= (year - 1996)):
        return "Gen Z"
    elif(age <= (year - 1977) and age > (year - 1995)):
        return "Gen Y"
    elif(age <= (year - 1965) and age > (year - 1976)):
        return "Gen X"
    else:
        return "Baby Boomer"

In [5]:
#Dictionaries fo the categorical variables for Demographic dataset
ridreth3 = {1 : 'mexican', 2 : 'other hispanic', 
            3 : 'non-hispanic White', 4 : 'non-hispanic black',
            6 : 'non-hispanic Asian', 7 : 'other race'}

ridstatr = {1 : 'interviewed only', 2: 'both'}

dmdecu2 ={1 : 'less than 9th', 2 : '9 to 11', 3:'hs graduate',
          4 : 'college', 5 : 'college graduate', 7 : 'refused',
          9 : 'dont know'}
 
dmdmartl = {1 : 'married', 2 : 'widowed', 3 : 'divorced',
            4 : 'separated', 5 : 'never married',
            6: 'living with partner', 77 : 'refused',
            99 : 'dont know'}
riagendr = {1 : 'male', 2: 'female'}

In [6]:
# Combining NHANES Demographic data from different years
def initial_data(nhanes_link):
    """
    Cleaning and Combining NHANES Demographic DataFrame 
    
    Parameters
    ----------
    nhanes_link : list
    Specifies the location for the demographic dataset(different years)

    Returns
    -------
    nhanes_data: Pandas Dataframe
    Gives the combined Dataframe as output
    
    """

    nhanes_data = pd.DataFrame()
    for i in range(len(nhanes_link)):
        data = pd.read_sas(nhanes_link[i])
        data = data.loc[:,['SEQN', 'RIDAGEYR', 'RIAGENDR','RIDRETH3',
          'DMDEDUC2', 'DMDMARTL', 'RIDSTATR',
          'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR']]
        #Adding the cohort column
        data['cohort'] = data['RIDAGEYR'].apply(cohort_def, data=nhanes_link[i])
        nhanes_data = nhanes_data.append(data)
    
    # Correcting the type of the different columns in the table
    for i in nhanes_data.columns:
        if(i == 'RIDAGEYR' or i == 'SDMVPSU' or i == 'SDMVSTRA'):
            nhanes_data[i] = nhanes_data[i].astype('int64')
        elif(i == 'RIDRETH3'):
            nhanes_data[i] = pd.Categorical(nhanes_data[i].replace(ridreth3))
        elif(i == 'RIAGENDR'):
            nhanes_data[i] = pd.Categorical(nhanes_data[i].replace(riagendr))
        elif(i == 'RIDSTATR'):
            nhanes_data[i] = pd.Categorical(nhanes_data[i].replace(ridstatr))
        elif(i == 'DMDEDUC2'):
            nhanes_data[i] = pd.Categorical(nhanes_data[i].replace(dmdecu2))
        elif(i == 'DMDMARTL'):
            nhanes_data[i] = pd.Categorical(nhanes_data[i].replace(dmdmartl))
        elif(i == 'cohort'):
            nhanes_data[i] = nhanes_data[i].astype('category')
        elif(i == 'WTMEC2YR' or i == 'WTINT2YR'):
            nhanes_data[i] = nhanes_data[i].astype('float64')
            
    nhanes_data = nhanes_data.rename(columns=
                                     {'RIDAGEYR':'age', 
                                      'RIDRETH3':'race',
                                      'RIAGENDR': 'gender',
                                      'DMDEDUC2':'education_level',
                                      'DMDMARTL':'marital', 
                                      'RIDSTATR':'interview_status',
                                      'SDMVPSU':'psuedo_PSU_var', 
                                      'SDMVSTRA':'psuedo_stratum_var', 
                                      'WTMEC2YR':'sample_weight_MEC', 
                                      'WTINT2YR':'sample_weight_int'})
        
    # Tidying and making the dataset clearer to read
    nhanes_data = nhanes_data.set_index('SEQN')
    return nhanes_data

In [7]:
nhanes_data = initial_data(nhanes)

In [8]:
print(nhanes_data.dtypes)

age                      int64
gender                category
race                  category
education_level       category
marital               category
interview_status      category
psuedo_PSU_var           int64
psuedo_stratum_var       int64
sample_weight_MEC      float64
sample_weight_int      float64
cohort                category
dtype: object


In [9]:
#Converting the nhanes_data to 'parquet' format
nhanes_data.to_parquet('Nhanes Data.parquet')

### b. Appending Oral and Dentition Data

**Meaning of variables:**  

**SEQN**: Respondent sequence number  
**OHDDESTS**: Dentition Status  
**Tooth counts (OHXxxTC)**: The Tooth Counts for each tooth (1-32)  
**Coronal cavities (OHXxxCTC)**:The Cavity Counts for each tooth (1-31)

In [10]:
# Taking out the columns for slicing
oral_rows = []
data = pd.read_sas(oral_dent[0])
for i in data.columns[0:64]:
    if(i == 'OHXIMP' or i == 'OHDEXSTS'):
        pass
    else:
        oral_rows.append(i)

/Users/anuraagramesh/opt/miniconda3/envs/stats507/lib/python3.9/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [11]:
#Dictionaries fo the categorical variables for Demographic dataset
tc = {1 : 'primary tooth',
      2 : 'permanent tooth',
      3 : 'dental implant',
      4 : 'not present',
      5 : 'root',
      9 : 'could not assess'}

# r = restorative
ctc = {'A' : 'restored primary tooth',
       'D' : 'sound primary tooth',
       'E' : 'missing - dental disease',
       'F' : 'restored permanent tooth',
       'J' : 'ppermanent root tip',
       'K' : 'primary tooth - surface condition',
       'M' : 'missing - other causes',
       'P' : 'disease - r',
       'Q' : 'other causes - r',
       'R' : 'other causes - fr',
       'S' : 'sound permanent tooth',
       'T' : 'permanent root - r',
       'U' : 'unerupted',
       'X' : 'other causes - f',
       'Y' : 'not assessed',
       'Z' : 'permanent tooth - surface condition'}

oddests={1 : 'complete',
         2 : 'partial',
         3 : 'not done'}

In [12]:
# Combining Oral and Dentition data from different years
def initial_data_oral(oral_link):
    """
    Cleaning and Combining NHANES Oral and Dentition DataFrame 
    
    Parameters
    ----------
    oral_link : list
    Specifies the location for the nhanes dataset(Different years)

    Returns
    -------
    oral_data: Pandas Dataframe
    Gives the combined Dataframe as output
    
    """
    
    oral_data = pd.DataFrame()
    for i in range(len(oral_link)):
        data = pd.read_sas(oral_link[i])
        data = data.loc[:, oral_rows]
        oral_data = oral_data.append(data)
    # Tidying and making the dataset clearer to read
    # Correcting the type of the different columns in the table
    for i in oral_rows:
        if('CTC' in i):
            oral_data[i] = oral_data[i].str.decode('utf-8') 
            oral_data[i] = pd.Categorical(oral_data[i].replace(ctc))
            oral_data = oral_data.rename(columns={i:'coronial_' + i[3:5]})
        elif('TC' in i):
            oral_data[i] = pd.Categorical(oral_data[i].replace(tc))
            oral_data = oral_data.rename(columns={i:'tooth_count_' + i[3:5]})
        elif(i == 'OHDDESTS'):
            oral_data[i] = pd.Categorical(oral_data[i].replace(oddests))
            oral_data = oral_data.rename(columns={i:'dentition_status'})
    oral_data = oral_data.set_index('SEQN')
    return oral_data

In [13]:
oral_data = initial_data_oral(oral_dent)

In [14]:
print(oral_data.dtypes)

dentition_status    category
tooth_count_01      category
tooth_count_02      category
tooth_count_03      category
tooth_count_04      category
                      ...   
coronial_27         category
coronial_28         category
coronial_29         category
coronial_30         category
coronial_31         category
Length: 61, dtype: object


In [15]:
#Converting the data to 'parquet' format
oral_data.to_parquet('Oral Data.parquet')

Both the datasets are saved in **parquet** format:  
- Nhanes Data: 'Nhanes Data.parquet'
- Oral Data: 'Oral Data.parquet'

### c. 

In [16]:
# Read NHANES data from parquet
nhanes_data1 = pd.read_parquet('Nhanes Data.parquet')
# Length of Nhanes dataset
print('Number of cases in NHANES data:', len(nhanes_data1))

# Read Oral Dentition data from parquet
oral_data1 = pd.read_parquet('Oral Data.parquet')
# Length of Oral Dentition dataset
print('Number of cases in NHANES data:', len(oral_data1))

Number of cases in NHANES data: 39156
Number of cases in NHANES data: 35909
